Imports


In [207]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
%matplotlib inline

Arrumando Preço

In [208]:
# Definição da função para arrumar o valor do preço
def arruma(preco):
    if pd.isnull(preco):
        return CellDF['PREÇO'].median()  # Retorna a mediana da coluna 'PREÇO' se o valor for nulo
    preco = preco.strip().replace('R$', '', 1) # Remove 'R$' do valor
    preco = preco.replace('.', '_') # Substitui pontos por underscores
    preco = preco.replace(',', '.') # Substitui vírgulas por pontos
    preco = preco.replace('_', '')  # Remove underscores restantes
    return np.float32(preco) # Retorna o valor convertido para ponto flutuante


# Dicionário para aplicar a função de conversão personalizada
sconverter = {
    'PREÇO': arruma,
}


# Leitura do arquivo CSV com aplicação da função de conversão
CellDF = pd.read_csv('resultado_parcial.csv', converters=sconverter)

# Exibir informações sobre o DataFrame
print(CellDF.info())

# Remoção das linhas em que o valor da coluna 'PREÇO' é maior ou igual a 9000
CellDF = CellDF.drop(CellDF[CellDF['PREÇO'] >= 9000].index)

# Exibir informações atualizadas sobre o DataFrame
CellDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431 entries, 0 to 430
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MARCA           429 non-null    object 
 1   MODELO          423 non-null    object 
 2   SISTEMA_OP      431 non-null    object 
 3   REDE            431 non-null    object 
 4   MEMORIA_INT     425 non-null    float64
 5   RAM             391 non-null    float64
 6   RESOL CAM TRAS  374 non-null    float64
 7   GARANTIA        399 non-null    float64
 8   PESO            429 non-null    float64
 9   PREÇO           431 non-null    float32
dtypes: float32(1), float64(5), object(4)
memory usage: 32.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 422 entries, 0 to 430
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MARCA           421 non-null    object 
 1   MODELO          414 non-null    object 


Dropando os nulos

In [209]:
# O método dropna() é usado para eliminar as linhas que possuem pelo menos um valor nulo.
dfamor = CellDF.dropna()

Tratando Modelos mal formados

In [210]:
dfamor['MARCA']=  dfamor['MARCA'].apply(lambda x: x.upper().strip()) 
dfamor['MARCA']=  dfamor['MARCA'].apply(lambda x: 'SAMSUNG' if x == 'GALAXY S22 PLUS' else x)
dfamor['MARCA']=  dfamor['MARCA'].apply(lambda x: 'SAMSUNG' if x.find('SAMSUNG') != -1 else x)
dfamor['MARCA']=  dfamor['MARCA'].apply(lambda x: 'MOTOROLA' if x.find('MOTOROLA') != -1 else x)

dfamor 

/tmp/ipykernel_39330/3817290528.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfamor['MARCA']=  dfamor['MARCA'].apply(lambda x: x.upper().strip())
/tmp/ipykernel_39330/3817290528.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfamor['MARCA']=  dfamor['MARCA'].apply(lambda x: 'SAMSUNG' if x == 'GALAXY S22 PLUS' else x)
/tmp/ipykernel_39330/3817290528.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

,MARCA,MODELO,SISTEMA_OP,REDE,MEMORIA_INT,RAM,RESOL CAM TRAS,GARANTIA,PESO,PREÇO
0,MOTOROLA,MOTO G 52,Android,4G,64.0,4.0,48.0,12.0,420.0,1089.000000
1,SAMSUNG,Galaxy A54,Android,5G,128.0,8.0,50.0,12.0,332.0,1859.000000
2,SAMSUNG,Galaxy A14,Android,5G,128.0,8.0,50.0,12.0,410.0,1129.000000
3,SAMSUNG,Galaxy A54,Android,5G,128.0,8.0,50.0,12.0,334.0,1859.000000
4,SAMSUNG,Galaxy A04e,Android,4G,64.0,4.0,13.0,12.0,284.0,759.000000
...,...,...,...,...,...,...,...,...,...,...
424,SAMSUNG,Galaxy S22+,Android,5G,128.0,8.0,50.0,12.0,327.0,6299.100098
425,SAMSUNG,Galaxy S22+,Android,5G,256.0,8.0,50.0,12.0,327.0,6749.100098
426,APPLE,iPhone 13,IOS,5G,256.0,6.0,12.0,12.0,1000.0,7547.000000
427,SAMSUNG,Galaxy S22+,Android,5G,128.0,8.0,50.0,12.0,413.0,4949.100098


In [211]:
dfamor['MARCA'].unique()
dfamor['PREÇO'].mean()

2827.552490234375

Variável categórica -> numérica

In [212]:
print(dfamor['MARCA'].unique())
print(dfamor['SISTEMA_OP'].unique())
print(dfamor['REDE'].unique())

['MOTOROLA' 'SAMSUNG' 'APPLE' 'ASUS' 'POSITIVO' 'XIAOMI' 'TCL' 'PHILCO'
 'INFINIX' 'BLUE' 'MULTILASER' 'NOKIA' 'LG' 'TECTOY' 'RED MOBILE'
 'HUSKY SPORTS' 'REALME']
['Android' 'IOS']
['4G' '5G' '3G']


In [213]:
# usado para realizar a codificação one-hot das colunas 'MARCA' e 'SISTEMA_OP'.
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False, categories='auto', dtype='int8') 

# criação de um novo DataFrame chamado dataframeenc contendo as features codificadas. 
# A função fit_transform() do encoder é aplicada às colunas 'MARCA' e 'SISTEMA_OP' do DataFrame 
# dfamor. O resultado é convertido em um DataFrame usando pd.DataFrame().
dataframeenc = pd.DataFrame(encoder.fit_transform(dfamor[['MARCA','SISTEMA_OP']]))

# Obtenção dos nomes das colunas codificadas usando e passando as etiquetas originais ('MARCA', 'SISTEMA_OP') como argumento.
columns = encoder.get_feature_names_out(['MARCA','SISTEMA_OP'])

#Atribuição dos nomes de colunas ao DataFrame dataframeenc usando dataframeenc.columns = columns.
dataframeenc.columns = columns

dataframeenc.index = dfamor.index

#Concatenação dos DataFrames dfamor e dataframeenc ao longo do eixo das colunas (axis=1)
#usando pd.concat(). O resultado é atribuído ao DataFrame dfamorepaixao
dfamorepaixao = pd.concat([dfamor, dataframeenc], axis=1)
dfamorepaixao = dfamorepaixao.drop(['MARCA', 'SISTEMA_OP', 'MODELO'],axis=1)

dfamorepaixao

/home/renanxm/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,REDE,MEMORIA_INT,RAM,RESOL CAM TRAS,GARANTIA,PESO,PREÇO,MARCA_APPLE,MARCA_ASUS,MARCA_BLUE,...,MARCA_PHILCO,MARCA_POSITIVO,MARCA_REALME,MARCA_RED MOBILE,MARCA_SAMSUNG,MARCA_TCL,MARCA_TECTOY,MARCA_XIAOMI,SISTEMA_OP_Android,SISTEMA_OP_IOS
0,4G,64.0,4.0,48.0,12.0,420.0,1089.000000,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,5G,128.0,8.0,50.0,12.0,332.0,1859.000000,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,5G,128.0,8.0,50.0,12.0,410.0,1129.000000,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,5G,128.0,8.0,50.0,12.0,334.0,1859.000000,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,4G,64.0,4.0,13.0,12.0,284.0,759.000000,0,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,5G,128.0,8.0,50.0,12.0,327.0,6299.100098,0,0,0,...,0,0,0,0,1,0,0,0,1,0
425,5G,256.0,8.0,50.0,12.0,327.0,6749.100098,0,0,0,...,0,0,0,0,1,0,0,0,1,0
426,5G,256.0,6.0,12.0,12.0,1000.0,7547.000000,1,0,0,...,0,0,0,0,0,0,0,0,0,1
427,5G,128.0,8.0,50.0,12.0,413.0,4949.100098,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [214]:
#Criação de um dicionário maping que mapeia os valores '3G', '4G' e '5G' 
# para os valores numéricos 0, 1 e 2, respectivamente.
maping = {
    '3G': 0,
    '4G': 1,
    '5G': 2
}

#Substituição dos valores na coluna 'REDE' do DataFrame dfamorepaixao 
#usando o método replace(). Os valores mapeados do dicionário maping são 
#atribuídos à coluna 'REDE' do DataFrame.
dfamorepaixao['REDE'] = dfamorepaixao['REDE'].replace(maping)

dfamorepaixao['REDE'].unique()

array([1, 2, 0])

In [215]:
dfamorepaixao.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 309 entries, 0 to 428
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   REDE                309 non-null    int64  
 1   MEMORIA_INT         309 non-null    float64
 2   RAM                 309 non-null    float64
 3   RESOL CAM TRAS      309 non-null    float64
 4   GARANTIA            309 non-null    float64
 5   PESO                309 non-null    float64
 6   PREÇO               309 non-null    float32
 7   MARCA_APPLE         309 non-null    int8   
 8   MARCA_ASUS          309 non-null    int8   
 9   MARCA_BLUE          309 non-null    int8   
 10  MARCA_HUSKY SPORTS  309 non-null    int8   
 11  MARCA_INFINIX       309 non-null    int8   
 12  MARCA_LG            309 non-null    int8   
 13  MARCA_MOTOROLA      309 non-null    int8   
 14  MARCA_MULTILASER    309 non-null    int8   
 15  MARCA_NOKIA         309 non-null    int8   
 16  MARCA_PH

Treino e Teste + Normalização

In [216]:
#Essas etapas preparam os conjuntos de dados de treinamento e teste para o treinamento e avaliação de um modelo de regressão

x = (dfamorepaixao.drop('PREÇO', axis = 1)).to_numpy()
y = (dfamorepaixao['PREÇO']).to_numpy()

xtreino, xteste, ytreino, yteste = train_test_split(x,y, train_size=0.7)

In [217]:
#Criação de um objeto Scaler do tipo MinMaxScaler(), que será
# usado para realizar a escala dos recursos (features)

Scaler = MinMaxScaler()
columns = xtreino
display(yteste)

#aplicação do método fit_transform() do objeto Scaler para ajustar e transformar
#os dados de treinamento (xtreino) em uma escala específica. Os dados transformados 
#são armazenados novamente em xtreino.
xtreino = Scaler.fit_transform(xtreino)
xteste = Scaler.transform(xteste)

# Criação de um novo objeto Scaleroutro do tipo MinMaxScaler(), que será usado para realizar a escala do target (y).
Scaleroutro = MinMaxScaler()

# Aplicação do método transform() do objeto Scaleroutro para transformar
# os dados de teste do target (yteste) com base no ajuste realizado nos dados de treinamento. Os dados são remodelados usando reshape(-1,1). Os dados transformados são armazenados em yteste.
ytreino = Scaleroutro.fit_transform(ytreino.reshape(-1,1))
yteste = Scaleroutro.transform(yteste.reshape(-1,1))


display(ytreino)

array([ 990.  , 1999.99, 1166.6 , 2339.19, 1999.99, 8999.1 ,  699.9 ,
       6999.  , 1290.9 , 1166.6 , 1999.99, 1349.99, 1199.9 , 6209.1 ,
        764.99, 1492.47, 3597.37,  854.99, 4949.1 , 4199.  , 1290.9 ,
       2659.05,  899.1 , 2743.  , 2531.  , 1492.47,  806.53,  848.9 ,
       5199.  , 3149.1 , 2599.2 ,  350.55, 1079.99,  759.  , 2623.95,
       5609.  , 3597.37, 4679.1 , 2116.25, 1169.09,  939.59, 2519.1 ,
       1166.6 , 5898.  , 3941.55,  296.1 , 2199.  , 3397.97,  759.  ,
       1399.99, 3419.1 , 4949.1 ,  989.  , 6759.  , 2024.1 , 2142.3 ,
       2050.92, 3599.99,  989.9 , 1399.99, 3059.1 , 2998.  , 4299.  ,
       1450.  , 1999.99, 1349.99, 6150.  , 1519.05,  577.99, 5299.  ,
       2575.  ,  899.99, 4299.  , 4799.  , 2189.38, 1691.99, 4949.  ,
        879.  , 1097.25, 1651.55,  899.1 , 1215.4 , 4409.1 , 1439.1 ,
       3149.99, 1161.8 , 7598.1 , 2699.99, 6209.1 ,  929.69, 1859.  ,
       3285.5 , 8567.  ], dtype=float32)

array([[0.06750461],
       [0.17801109],
       [0.7697428 ],
       [0.9601917 ],
       [0.9150187 ],
       [0.19410066],
       [0.22292055],
       [0.17111522],
       [0.06220895],
       [0.21774001],
       [0.07795778],
       [0.18710004],
       [0.07440047],
       [0.45614687],
       [0.30580902],
       [0.0985878 ],
       [0.91940254],
       [0.12505455],
       [0.32663357],
       [0.09501898],
       [0.79760253],
       [0.05137589],
       [0.6788072 ],
       [0.23338406],
       [0.06519064],
       [0.17121768],
       [0.51303023],
       [0.5122589 ],
       [0.08097285],
       [0.6891683 ],
       [0.09397137],
       [0.07208651],
       [0.49344784],
       [0.6891683 ],
       [0.19412829],
       [0.03652389],
       [0.10905132],
       [0.50266916],
       [0.8915428 ],
       [0.22810109],
       [0.21532242],
       [0.09185195],
       [0.0429489 ],
       [0.14876987],
       [0.5337524 ],
       [0.06220895],
       [0.2540037 ],
       [0.125

In [218]:
# Criação de um objeto model do tipo LinearRegression(), que será usado para treinar um modelo de regressão linear
model = LinearRegression()

model.fit(xtreino, ytreino)

# Geração de previsões para os dados de teste usando model.predict(xteste).
#  As previsões são armazenadas na variável prediction.
prediction = model.predict(xteste)

# Inversão da escala dos valores de teste original (yteste) usando Scaleroutro.
# inverse_transform(yteste). Isso é feito para retornar os valores à escala original antes da normalização.
yteste = Scaleroutro.inverse_transform(yteste)

prediction = Scaleroutro.inverse_transform(prediction)

#Cálculo do erro médio quadrático entre os valores de teste original (yteste) e as previsões 
#(prediction) usando mean_squared_error(yteste, prediction).

mean_squared_error(yteste, prediction)


print(np.sqrt(mean_squared_error(yteste, prediction)))

1088.8649491451813
